# Dense Neural Network
A dense NN is also called a Fully Connected (FC) or Multi-Layer Perceptron (MLP).   

## Architecture
The network consists of layers. 
The network architecture is: Input Layer | Internal Layer(s) | Output Layer.
A deep NN has many internal layers.

Each layer consists of Neurons.
The input layer should have one neuron per feature of the input vector.
The output layer should have one neuron per label that needs to be predicted.

In the MLP architecture, 
neurons have no connections to other neurons in the same layer,
but they are connected to every neuron in the layers immediately before and after.

Neurons consist of Weighted Inputs | Summation | Activation Function | Output.
With non-linear activations, the network can model any non-linear function.
Each neuron also has one weighted input called the bias.
The bias is merely for scaling the output, 
equivalent to the y-intercept b in the linear equation y=mx+b.

## Training
Trainable parameters:  
The model as trainable weights on every edge.
The weights are initialized randomly (e.g. by the glorot_uniform algorithm).

One forward calculation:  
The model is shown one data instance from the labeled training set.
Using the weights and activations, the output scores are computed.

One back propagation:   
For each labeled training instance, the actual scores are known.
A loss function is applied to the calculated as actual minus difference.
The partial derivative of the loss, with respect to each input, provides a gradient.
The gradient tells the amount and direction that each input contributed to the loss.
Using partial derivatives of the activations,
the total loss is distributed, such that every neuron accepts some portion of the loss. 
Finally, the weight of each neuron's output is updated
in the opposite direction of the gradient, so as to reduce the loss.
The size of the update can be controlled by a learn rate parameter.

Gradient descent:   
The back propagation can apply to the loss over any number of instances, called a batch.
If the batch size is 1, and each next training instance is chosen randomly,
that is called "stochastic gradient descent."
If the batch size equals the whole training set,
that is called "batch gradient descent" but it is computationally prohibitive.
In keras, the default batch size is 32, 
and instances within each batch are used in random order.

Epoch:   
When all the training samples have been used once, the model has been trained for one epoch.
It is usually necessary to train a model for many epochs.

## Evaluation
Validiation cycles:   
We train a model on a training set.
We evaluate the model on a separate validation set.
In cycles, we adjust the model architecture and hyperparameters
until the validation accuracy looks good.
Finally, we measure the true accuracy on the previously-unused test data.

Test set:   
Some portion of the data, maybe 20%, is set aside for post-training evaluation.
It is very important to avoid looking within the test set.
Otherwise, the model accuracy could be artificially and inadvertently inflated.

Validation set:   
Some portion of the remaining data, maybe 20%, is set aside for hyperparameter tuning.
For instance, should the deep neural network have 10 layers or 20?
The model gets trained and evaluated and then tuned in cycles.
It is important to avoid using any validation data for training, and vice-versa.
Otherwise, the validation accuracy could be inflated.

Training set:   
After removing the test and validation data, the training set is what remains.

Cross validation:   
This is used to alleviate a data shortage,
when we would rather not have to set aside a validation set.
In 5-fold cross-validation, we make 5 mutually exclusive partions,
of 80% train and 20% valid. 
Each data instaince will be used 4 times for training and 1 time for validation.
The total validation accuracy is the mean of 5 rounds of training & validation.
After model tuning is complete, the full dataset is used to retrain the model.
The model thus trained is then evaluated on the previously-unseen test set.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2022-09-03 18:40:18.480499
Python 3.10.0


In [2]:
from tensorflow import keras
from keras import layers
from sklearn.model_selection import StratifiedKFold

## The Dataset
Iris:   
We'll use the Iris dataset,
famous because it was used in a paper by R.A. Fisher.
There are 150 data instances, each a flower with 4 numeric features.
Each instance is one of 3 species, labled 0, 1, 2.

Shuffle:  
The data are provided in order (0s first, 2s last).
We must avoid training on 0s and testing on 2s, so we shuffle the rows.
In our experiments, this was critical.

Normalize:   
Some features have larger values and larger variances than others.
We must not let the model infer feature importance from those attributes.
We use a library function that applies mean-center and unit-variance.
This made no difference in our experiments on this data, but it could have.

In [3]:
from sklearn import datasets
from sklearn.utils import shuffle
iris = datasets.load_iris()
print('shape',iris.data.shape)
print('features',iris.feature_names)
print('labels',iris.target_names)
X = iris.data
y = iris.target
# At start, all species=0 come first and all species=2 come last.
# Important to shuffle rows before doing train/test split and train/valid split.
# Another way is using the sklearn train_test_split class.
X, y = shuffle(X, y, random_state=0)
print('X shape',X.shape)
print('y shape',y.shape)
num_instances = X.shape[0]
num_features =  X.shape[1]

shape (150, 4)
features ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
labels ['setosa' 'versicolor' 'virginica']
X shape (150, 4)
y shape (150,)


In [4]:
X

array([[5.8, 2.8, 5.1, 2.4],
       [6. , 2.2, 4. , 1. ],
       [5.5, 4.2, 1.4, 0.2],
       [7.3, 2.9, 6.3, 1.8],
       [5. , 3.4, 1.5, 0.2],
       [6.3, 3.3, 6. , 2.5],
       [5. , 3.5, 1.3, 0.3],
       [6.7, 3.1, 4.7, 1.5],
       [6.8, 2.8, 4.8, 1.4],
       [6.1, 2.8, 4. , 1.3],
       [6.1, 2.6, 5.6, 1.4],
       [6.4, 3.2, 4.5, 1.5],
       [6.1, 2.8, 4.7, 1.2],
       [6.5, 2.8, 4.6, 1.5],
       [6.1, 2.9, 4.7, 1.4],
       [4.9, 3.6, 1.4, 0.1],
       [6. , 2.9, 4.5, 1.5],
       [5.5, 2.6, 4.4, 1.2],
       [4.8, 3. , 1.4, 0.3],
       [5.4, 3.9, 1.3, 0.4],
       [5.6, 2.8, 4.9, 2. ],
       [5.6, 3. , 4.5, 1.5],
       [4.8, 3.4, 1.9, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [6.2, 2.8, 4.8, 1.8],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.8, 1.9, 0.4],
       [6.2, 2.9, 4.3, 1.3],
       [5. , 2.3, 3.3, 1. ],
       [5. , 3.4, 1.6, 0.4],
       [6.4, 3.1, 5.5, 1.8],
       [5.4, 3. , 4.5, 1.5],
       [5.2, 3.5, 1.5, 0.2],
       [6.1, 3. , 4.9, 1.8],
       [6.4, 2

In [5]:
y

array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1,
       0, 0, 2, 0, 0, 1, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 0, 2, 0,
       0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 1, 1,
       1, 2, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 0, 2, 2, 2, 2, 0,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 2, 1, 2, 1, 0, 2, 0, 2, 0, 0, 2, 0, 2, 1, 1, 1, 2, 2,
       1, 1, 0, 1, 2, 2, 0, 1, 1, 1, 1, 0, 0, 0, 2, 1, 2, 0])

Without normalization, the feature with biggest variance dominates.  
Unfortunately, on these data, normalization reduces the attainable accuracy.   
We use the Keras Normalization class:
1. Constructor takes axis, mean, var. By default, axis = -1 (the last axis), mean=0, var=1.
1. The adapt() method tunes the scaling parameters to the given data.
1. From then on, the object scales any given data.
1. Importantly, we adapt() to the training data and apply to the test data.

In [6]:
# Normalize on half the data. This isn't exactly fair, as some test data is included.
X_train = X[:75] 
normalizer = layers.Normalization() 
normalizer.adapt(X_train)
X = normalizer(X)
X

2022-09-03 18:40:54.475680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(150, 4), dtype=float32, numpy=
array([[-0.04237568, -0.4948195 ,  0.7840797 ,  1.7010827 ],
       [ 0.23398574, -1.8361979 ,  0.09298381, -0.33556128],
       [-0.45691848,  2.6350634 , -1.5405157 , -1.4993577 ],
       [ 2.0303369 , -0.2712561 ,  1.5380026 ,  0.82823515],
       [-1.1478227 ,  0.84655935, -1.4776887 , -1.4993577 ],
       [ 0.6485285 ,  0.622996  ,  1.3495218 ,  1.8465571 ],
       [-1.1478227 ,  1.0701222 , -1.6033425 , -1.3538831 ],
       [ 1.2012514 ,  0.17586967,  0.532772  ,  0.39181152],
       [ 1.3394327 , -0.4948195 ,  0.5955991 ,  0.24633694],
       [ 0.37216645, -0.4948195 ,  0.09298381,  0.10086234],
       [ 0.37216645, -0.9419458 ,  1.0982141 ,  0.24633694],
       [ 0.78670925,  0.39943308,  0.40711832,  0.39181152],
       [ 0.37216645, -0.4948195 ,  0.532772  , -0.04461209],
       [ 0.92489   , -0.4948195 ,  0.46994513,  0.39181152],
       [ 0.37216645, -0.2712561 ,  0.532772  ,  0.24633694],
       [-1.2860034 ,  1.2936851 , -

## Model Hyperparameters
Here we decide number of layers, number of neurons per layer, and activation functions.
Deciding this is an artform. 
In fact, we played with several settings before settling on the ones shown here. 

Keras offers 2 means of building a model.  
1. The [Sequential class](https://keras.io/api/models/sequential/#sequential-class) is simple. It has an add(layer) method. It maintains layers in the order they were added. It allows linear models only. Every layer has one input and one output.
1. The [Functional API](https://keras.io/guides/functional_api/) is complex. It allows DAG models. Layers can be skipped, reused, and multiply connected. We use the Functional API even though our model is linear in this example.

The Keras Dense layer class has these constructor options.
1. num units
1. activation. Default=linear. Choices are relu, elu, softplus (smooth relu), softmax, sigmoid, tanh, exponential. There is an API so you can build your own.
1. use_bias. Default=True
1. kernel_initializer. Default="glorot_uniform"
1. regularizers for the bias, kernel, and activity. Default=None.

In [7]:
inputs = keras.Input(shape=(num_features,))
inputs

<KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'input_1')>

In [8]:
# Keras output layer raises exception unless #neurons == #labels in Y.
dense1 =  layers.Dense(8, activation="relu")   (inputs)
dense2 =  layers.Dense(8, activation="relu")   (dense1)
outputs = layers.Dense(3)                      (dense2)
outputs

<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'dense_2')>

## Trainable Parameters
Here we explain the number of trainable parameters that are shown below.
1. Dense_0: (4 input + 1 bias) * 8 out = 40 
1. Dense_0: (8 input + 1 bias) * 8 out = 72 
1. Dense_2: (8 input + 1 bias) * 3 out = 27

In [9]:
model = keras.Model(inputs=inputs, outputs=outputs, name="MLP_1")
model.summary()

Model: "MLP_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 8)                 40        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 3)                 27        
                                                                 
Total params: 139
Trainable params: 139
Non-trainable params: 0
_________________________________________________________________


## Cross Validation
We evaluate the model by 5-fold cross validation.
We use "stratified" KFold so that each validation set
has approximately the same number of each label (flower species).

Within each loop, the cricial steps are:
1. Re-compile the model. Otherwise we would continously improve the same model.
2. Fit the model to the training data by adjusting parameters. In Keras, model.fit() is made to be customized, but we use it as is. By default, [model.fit()](https://keras.io/api/models/model_training_apis/#fit-method)
has shuffle=True within each batch.  
3. Evaluate the model on the validation data.

In [13]:
UNSEEN_PORTION = 0.10
split_point = int ( X.shape[0] * (1-UNSEEN_PORTION) )
X_unseen = np.array(X)[split_point:]
y_unseen = np.array(y)[split_point:]
X_seen   = np.array(X)[:split_point]
y_seen   = np.array(y)[:split_point]

In [14]:
EPOCHS=100
fold=1
skf = StratifiedKFold(n_splits=5)
history_vector=[]
valid_vector=[]
for train, valid in skf.split(X_seen,y_seen):
    print(type(X))
    print(type(train))
    X_train = np.array(X_seen)[train]
    y_train = np.array(y_seen)[train]
    X_valid = np.array(X_seen)[valid]
    y_valid = np.array(y_seen)[valid]
    model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),metrics=["accuracy"])
    history = model.fit(X_train, y_train, batch_size=25, epochs=EPOCHS)
    valid_scores = model.evaluate(X_valid, y_valid)
    print("Fold", fold,"Valid accuracy:",valid_scores[1],"Valid loss:", valid_scores[0])
    fold += 1
    history_vector.append(history)
    valid_vector.append(valid_scores)

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
Epoch 1/100
5/5 [==============================] - 1s 3ms/step - loss: 1.0914 - accuracy: 0.3611
Epoch 2/100
5/5 [==============================] - 0s 5ms/step - loss: 1.0500 - accuracy: 0.3889
Epoch 3/100
5/5 [==============================] - 0s 6ms/step - loss: 1.0232 - accuracy: 0.4259
Epoch 4/100
5/5 [==============================] - 0s 5ms/step - loss: 0.9982 - accuracy: 0.4630
Epoch 5/100
5/5 [==============================] - 0s 5ms/step - loss: 0.9762 - accuracy: 0.5185
Epoch 6/100
5/5 [==============================] - 0s 6ms/step - loss: 0.9545 - accuracy: 0.5463
Epoch 7/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9369 - accuracy: 0.5833
Epoch 8/100
5/5 [==============================] - 0s 3ms/step - loss: 0.9194 - accuracy: 0.6111
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9032 - accuracy: 0.6296
Epoch 10/100
5/5 [==============================]

5/5 [==============================] - 0s 4ms/step - loss: 0.5045 - accuracy: 0.8704
Epoch 84/100
5/5 [==============================] - 0s 4ms/step - loss: 0.5001 - accuracy: 0.8704
Epoch 85/100
5/5 [==============================] - 0s 5ms/step - loss: 0.4961 - accuracy: 0.8704
Epoch 86/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4917 - accuracy: 0.8704
Epoch 87/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4892 - accuracy: 0.8796
Epoch 88/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4843 - accuracy: 0.8796
Epoch 89/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4817 - accuracy: 0.8889
Epoch 90/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4774 - accuracy: 0.8796
Epoch 91/100
5/5 [==============================] - 0s 4ms/step - loss: 0.4735 - accuracy: 0.9074
Epoch 92/100
5/5 [==============================] - 0s 3ms/step - loss: 0.4702 - accuracy: 0.8981
Epoch 93/100
5/5 [===============

5/5 [==============================] - 0s 4ms/step - loss: 0.1137 - accuracy: 0.9815
Epoch 65/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1109 - accuracy: 0.9815
Epoch 66/100
5/5 [==============================] - 0s 4ms/step - loss: 0.1079 - accuracy: 0.9815
Epoch 67/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1055 - accuracy: 0.9815
Epoch 68/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1030 - accuracy: 0.9815
Epoch 69/100
5/5 [==============================] - 0s 3ms/step - loss: 0.1004 - accuracy: 0.9815
Epoch 70/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0996 - accuracy: 0.9815
Epoch 71/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0961 - accuracy: 0.9815
Epoch 72/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0944 - accuracy: 0.9815
Epoch 73/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0923 - accuracy: 0.9815
Epoch 74/100
5/5 [===============

5/5 [==============================] - 0s 3ms/step - loss: 0.0702 - accuracy: 0.9815
Epoch 46/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0710 - accuracy: 0.9815
Epoch 47/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0684 - accuracy: 0.9815
Epoch 48/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0688 - accuracy: 0.9815
Epoch 49/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0685 - accuracy: 0.9815
Epoch 50/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0674 - accuracy: 0.9815
Epoch 51/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0681 - accuracy: 0.9815
Epoch 52/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0671 - accuracy: 0.9815
Epoch 53/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0661 - accuracy: 0.9815
Epoch 54/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0665 - accuracy: 0.9815
Epoch 55/100
5/5 [===============

5/5 [==============================] - 0s 2ms/step - loss: 0.0489 - accuracy: 0.9815
Epoch 27/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0495 - accuracy: 0.9815
Epoch 28/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0493 - accuracy: 0.9815
Epoch 29/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0491 - accuracy: 0.9815
Epoch 30/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0492 - accuracy: 0.9815
Epoch 31/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0509 - accuracy: 0.9815
Epoch 32/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0496 - accuracy: 0.9907
Epoch 33/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0486 - accuracy: 0.9815
Epoch 34/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0490 - accuracy: 0.9815
Epoch 35/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0498 - accuracy: 0.9815
Epoch 36/100
5/5 [===============

5/5 [==============================] - 0s 3ms/step - loss: 0.0372 - accuracy: 0.9907
Epoch 8/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0373 - accuracy: 0.9907
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 0.0376 - accuracy: 0.9907
Epoch 10/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0365 - accuracy: 0.9907
Epoch 11/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0365 - accuracy: 0.9907
Epoch 12/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0366 - accuracy: 0.9907
Epoch 13/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0363 - accuracy: 0.9907
Epoch 14/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0358 - accuracy: 0.9907
Epoch 15/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0368 - accuracy: 0.9907
Epoch 16/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0368 - accuracy: 0.9907
Epoch 17/100
5/5 [=================

5/5 [==============================] - 0s 3ms/step - loss: 0.0296 - accuracy: 0.9907
Epoch 91/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0297 - accuracy: 0.9907
Epoch 92/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0296 - accuracy: 0.9907
Epoch 93/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0301 - accuracy: 0.9907
Epoch 94/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0290 - accuracy: 0.9907
Epoch 95/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0291 - accuracy: 0.9907
Epoch 96/100
5/5 [==============================] - 0s 3ms/step - loss: 0.0293 - accuracy: 0.9907
Epoch 97/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0295 - accuracy: 0.9907
Epoch 98/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0292 - accuracy: 0.9907
Epoch 99/100
5/5 [==============================] - 0s 2ms/step - loss: 0.0287 - accuracy: 0.9907
Epoch 100/100
1/1 [==============

In [15]:
for pair in valid_vector:
    print('Validation acc =',pair[1],'Validation loss =',pair[0])

Validation acc = 0.8148148059844971 Validation loss = 0.46995216608047485
Validation acc = 0.8518518805503845 Validation loss = 0.2636967897415161
Validation acc = 1.0 Validation loss = 0.015086145140230656
Validation acc = 1.0 Validation loss = 0.011959697119891644
Validation acc = 0.9629629850387573 Validation loss = 0.04753085598349571


In [ ]:
# TO DO: TEST ACC